## Parameters and variables

In [12]:
#Notebook parameters
#zoneAnalysis = "Japan"
zoneAnalysis = "Gulf of Mexico"

In [13]:
#usefull global variables
maxLon, minLon, maxLat, minLat = 0,0,0,0
minYear = 1998
maxYear = 2015

if (zoneAnalysis == "Gulf of Mexico"):
    maxLon = 282
    minLon = 256
    maxLat = 32
    minLat = 18
else:
    maxLon = 145
    minLon = 120
    maxLat = 40
    minLat = 20

## Initialisation & Data loading

In [1]:
!pip install basemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.5 which is incompatible.


In [2]:
%matplotlib inline
%pylab inline
from mpl_toolkits.basemap import Basemap
import sklearn
import pandas as pd
from scipy.stats.mstats import zscore
import warnings
warnings.filterwarnings("ignore") # disable warnings
pylab.rcParams['figure.figsize']=(15,15) # graph size

Populating the interactive namespace from numpy and matplotlib


In [3]:
# function to plot images  
def plot_im(lon,lat,im,size_points,var_name):
    
    # transform to arrays (just in case)
    lon=array(lon)
    lat=array(lat)
    im=array(im)
    
    if max(lon)-min(lon)<100:
      # Mercator projection (for small zone)
      m=Basemap(projection='merc',llcrnrlat=nanmin(lat),urcrnrlat=nanmax(lat),\
                llcrnrlon=nanmin(lon),urcrnrlon=nanmax(lon),lat_0=(nanmax(lat)+nanmin(lat))*0.5,\
                lon_0=(nanmax(lon)+nanmin(lon))*0.5,resolution='l')
    else:
      # Orthogonal projection (for large zone)
      m=Basemap(projection='robin',lat_0=0,lon_0=0,resolution='l')
    # you can use other projections (see https://matplotlib.org/basemap/users/mapsetup.html)
    
    # transform (lon,lat) to (x,y)
    x,y=m(lon,lat)

    # plot
    #im=ma.masked_where(isnan(im),im)
    res=m.scatter(x,y,size_points,im,'o',alpha=1,cmap='jet',lw=0)
    m.drawcoastlines()
    m.fillcontinents()
    parallels = linspace(nanmin(lat),nanmax(lat),15)
    meridians = linspace(nanmin(lon),nanmax(lon),15)
    #m.drawparallels(parallels,labels=[1,0,0,1],fontsize=10)
    #m.drawmeridians(meridians,labels=[1,0,0,1],fontsize=10)
    cb=m.colorbar(res,location="right")
    cb.set_label(var_name,fontsize=15)
    
# function to plot time series
def plot_ts(time,ts,line_type,var_name):

    # plot
    plot_date(time,ts,line_type)
    xlabel('Time',fontsize=15)
    ylabel(var_name,fontsize=15)

**Connection to the GCP:**

First, we have to connect to the Google Cloud Platform. Enter the login "bigdataocean2020@gmail.com" and password "bdoimt2023". Do it only once. We will maybe need an authentication: contact me at pierre.tandeo@imt-atlantique.fr.

In [4]:
import os
os.environ['USE_AUTH_EPHEM'] = '0'

from google.colab import auth
auth.authenticate_user()

In [17]:
params = {"min_lat": minLat, "max_lat": maxLat, "min_lon": minLon, "max_lon": maxLon}

### SST & SSH request to bigquery database

In [20]:
%%bigquery output --project alert-ground-261008 --params $params
SELECT lon, lat, time, AVG(sst) AS sst, AVG(ssh) AS ssh
FROM bdo2020.bdo2020.1998_2015
WHERE lon>@min_lon AND lon<@max_lon AND lat>@min_lat AND lat<@max_lat
GROUP BY lon, lat, time

Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
# saving the query
output.to_csv("data/sst-ssh-1998-2015-"+zoneAnalysis+".csv", index=False)

### IBTRACS data loading

In [24]:
#data loading from zone files
cyclonesData: pd.DataFrame
if (zoneAnalysis == "Japan"):
    cyclonesData = pd.read_csv("data/ibtracs-japan-1998-2015.csv")
else:
    cyclonesData = pd.read_csv("data/ibtracs-gulf of mexico-1998-2015.csv")

#time conversion
cyclonesData.ISO_TIME = pd.to_datetime(cyclonesData.ISO_TIME)
cyclonesData["NSEC_TIME"] = cyclonesData.NSEC_TIME.astype("int64")

## Data formating

In [66]:
output.drop(columns=["DAY_TIME"], inplace=True)

In [67]:
output[["lon", "lat"]] = output[["lon", "lat"]].round(1)
output.rename(columns={"lon": "LON", "lat": "LAT", "time": "DAY_TIME"}, inplace=True)
output

,LON,LAT,DAY_TIME,sst,ssh
0,262.9,22.9,731568.0,24.45,0.2469
1,278.4,18.4,731008.0,28.80,0.7054
2,279.1,20.4,730991.0,28.20,0.8500
3,278.1,20.9,732589.0,29.85,0.8531
4,278.4,20.1,732025.0,26.40,0.6866
...,...,...,...,...,...
14936123,271.9,25.4,729994.0,29.40,0.4028
14936124,271.9,26.9,730754.0,28.65,0.2783
14936125,271.9,27.6,730305.0,28.65,0.2542
14936126,271.9,28.6,729664.0,24.90,0.0785


In [68]:
cyclonesData["DAY_TIME"] = (cyclonesData["ISO_TIME"].dt.dayofyear + (cyclonesData["ISO_TIME"].dt.year*365.25).round())
cyclonesData["CYCLONE_PRESENT"] = 1
cyclonesData[["LAT", "LON"]] = cyclonesData[["LAT", "LON"]].round(1)
cyclonesData

,LAT,LON,SID,USA_WIND,CATEGORY,NSEC_TIME,ISO_TIME,DAY_TIME,CYCLONE_PRESENT
0,25.3,267.7,1998233N25268,25.0,TD,903679200000000000,1998-08-21 06:00:00,730003.0,1
1,25.3,266.9,1998233N25268,27.0,TD,903690000000000000,1998-08-21 09:00:00,730003.0,1
2,25.4,266.2,1998233N25268,30.0,TD,903700800000000000,1998-08-21 12:00:00,730003.0,1
3,25.6,265.8,1998233N25268,35.0,TS,903711600000000000,1998-08-21 15:00:00,730003.0,1
4,26.0,265.5,1998233N25268,40.0,TS,903722400000000000,1998-08-21 18:00:00,730003.0,1
...,...,...,...,...,...,...,...,...,...
3154,31.0,262.7,2015167N27266,30.0,TD,1434531600000000000,2015-06-17 09:00:00,736147.0,1
3155,31.7,262.6,2015167N27266,30.0,TD,1434542400000000000,2015-06-17 12:00:00,736147.0,1
3156,21.6,256.2,2015293N13266,50.0,TS,1445666400000000000,2015-10-24 06:00:00,736276.0,1
3157,22.5,256.9,2015293N13266,37.0,TS,1445677200000000000,2015-10-24 09:00:00,736276.0,1


In [72]:
finalData = pd.merge(left=output, right=cyclonesData[['LON', 'LAT', "DAY_TIME", "CYCLONE_PRESENT"]], how="left", on=['LON', 'LAT','DAY_TIME'])

In [73]:
finalData

,LON,LAT,DAY_TIME,sst,ssh,CYCLONE_PRESENT
0,262.9,22.9,731568.0,24.45,0.2469,NaN
1,278.4,18.4,731008.0,28.80,0.7054,NaN
2,279.1,20.4,730991.0,28.20,0.8500,NaN
3,278.1,20.9,732589.0,29.85,0.8531,NaN
4,278.4,20.1,732025.0,26.40,0.6866,NaN
...,...,...,...,...,...,...
14936133,271.9,25.4,729994.0,29.40,0.4028,NaN
14936134,271.9,26.9,730754.0,28.65,0.2783,NaN
14936135,271.9,27.6,730305.0,28.65,0.2542,NaN
14936136,271.9,28.6,729664.0,24.90,0.0785,NaN


In [74]:
print(len(finalData), len(output), len(cyclonesData))

14936138 14936128 3159


In [76]:
finalData.CYCLONE_PRESENT.count()

246